In [ ]:
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 9.5 MB/s 


In [ ]:
pip install bert-tensorflow

In [ ]:
pip install tokenization

In [29]:
import gc
from matplotlib import rcParams, pyplot as plt
import numpy as np
import pandas as pd
import os
from pathlib import Path
import re
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import sys
sys.path.append('../src')

import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.backend import clear_session
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import tensorflow_hub as hub
from bert.tokenization import FullTokenizer
import tokenization
import warnings 
warnings.filterwarnings(action='ignore')

In [ ]:
rcParams['figure.figsize'] = (16,8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)
warnings.simplefilter('ignore')

In [ ]:
# bert tokenizer load

module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
target_col = "target"
n_fold = 5
n_class = 5
seed = 42

In [13]:
algo_name = 'bert'
max_len = 100
feature_name = f'n{max_len}'
model_name = f'{algo_name}_{feature_name}'
sub_file = f'{model_name}.csv'

In [31]:
bert_layer = hub.KerasLayer(module_url, trainable=True)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

In [39]:
def create_tokenizer_from_hub_module(bert_path):
    """Get the vocab file and casing info from the Hub module."""
    bert_module =  hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

In [40]:
BERT_PATH = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
MAX_SEQ_LENGTH = 512

In [42]:
import tensorflow.compat.v1 as tf

In [ ]:
tokenizer = create_tokenizer_from_hub_module(bert_path=BERT_PATH)